In [1]:
# LOD determination for LC-ICPMS method 

from matplotlib.backends.backend_pdf import PdfPages

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

import os
from tempfile import tempdir
from time import time
from turtle import color
from unittest.mock import NonCallableMagicMock
import pandas as pd
import numpy as np
import warnings
import math
import re

import seaborn as sns
warnings.filterwarnings("ignore")

import sys
sys.path.append("./")

from pathlib import Path
import matplotlib.pyplot as plt
from copy import copy 


dir = '/Users/christiandewey/Library/CloudStorage/GoogleDrive-christian.w.dewey@gmail.com/My Drive/manuscripts/dewey-boiteau_LC-ICPMS/LOD/BIOZEN'

flist=os.listdir(dir)
os.chdir(dir)



In [20]:

def getPeakArea(df):

    cnames = df.columns.values.tolist()

    metal = cnames[1]

    t = cnames[0]

    summed_area = 0
    timeDelta = 0
    totalTime = 0
    me_col_ind = 1 # = df.columns.get_loc(metal)
    for i in range(0, len(df)-1):

        icp_1 = df.iloc[i,me_col_ind] # cps
        icp_2 = df.iloc[i+1,me_col_ind] 
        min_height = min([icp_1,icp_2])
        max_height = max([icp_1,icp_2])
        
        timeDelta = (df.iloc[i+1,me_col_ind - 1] - df.iloc[i,me_col_ind - 1])/60 # minutes; time is always to left of metal signal

        
        rect_area = timeDelta * min_height
        top_area = timeDelta * (max_height - min_height) * 0.5
        An = rect_area + top_area

        summed_area = summed_area + An  # area =  cps * sec = counts
        totalTime = totalTime + timeDelta
    return summed_area

In [ ]:
metals = ['55Mn', '59Co','60Ni','66Zn','111Cd','208Pb','63Cu', '56Fe', '115In']

f50ppb = {}
f100ppb = {}
f200ppb = {}

for f in flist:
    
    if '.csv' in f:

        metal = '115In'
        
        df = pd.read_csv(f,sep=';',header=1)

        if '50ppb' in f:
            t = 'Time ' + metal
            df = df[[t,metal]]
            area = getPeakArea(df)
            f50ppb[metal].append(area)
        
        elif '100ppb' in f:
            t = 'Time ' + metal
            df = df[[t,metal]]
            area = getPeakArea(df)
            f100ppb[metal].append(area)

        elif '200ppb' in f:
            t = 'Time ' + metal
            df = df[[t,metal]]
            area = getPeakArea(df)
            f200ppb[metal].append(area)

In [21]:
metals = ['55Mn', '59Co','60Ni','66Zn','111Cd','208Pb','63Cu', '56Fe', '115In']

c50ppb = {metal:[] for metal in metals}
c100ppb = {metal:[] for metal in metals}
c200ppb = {metal:[] for metal in metals}



for f in flist:
    
    if '.csv' in f:

        for metal in metals:
        
            df = pd.read_csv(f,sep=';',header=1)

            if '50ppb' in f:
                t = 'Time ' + metal
                df = df[[t,metal]]
                area = getPeakArea(df)
                if metal == '115In':
                    print(f, area)
                c50ppb[metal].append(area)
            
            elif '100ppb' in f:
                t = 'Time ' + metal
                df = df[[t,metal]]
                area = getPeakArea(df)
                c100ppb[metal].append(area)

            elif '200ppb' in f:
                t = 'Time ' + metal
                df = df[[t,metal]]
                area = getPeakArea(df)
                c200ppb[metal].append(area)

cwd_220505_13_zdv_50ppb_50uL.csv 31096.466618615632
cwd_220505_15_zdv_50ppb_50uL.csv 31351.823851542205
pwb_220125_50ppb_biozen1.7peptide_30min_50uL.csv 37634.92513359223
cwd_220110_day2_5_ZDV_50ppb_std_50uL.csv 32754.89815697196
cwd_220112_day2a_7_zdv_50ppb_50uL.csv 30522.03471953741
cwd_220505_16_zdv_50ppb_50uL.csv 32071.018438332463
cwd_220505_14_zdv_50ppb_50uL.csv 31406.79954750798
cwd_220505_12_zdv_50ppb_50uL.csv 31042.285643835003
cwd_220211_day1_6_zdv_50ppb_stdmix_50uL.csv 33381.62772927759
cwd_220204_day1_6_zdv_50ppb_50uL.csv 36310.42029272423


In [7]:
c50ppb['115In']

[31096.466618615632,
 31351.823851542205,
 37634.92513359223,
 32754.89815697196,
 30522.03471953741,
 32071.018438332463,
 31406.79954750798,
 31042.285643835003,
 33381.62772927759,
 36310.42029272423]

In [ ]:

#icpms_file = '/Users/christiandewey/Library/CloudStorage/Box-Box/Boiteau Lab/Mass Spec Data/Keck iCAP Q/2022/2022-09/Tillamook, Sept 2022/spring_ENV.csv'

icpms = pd.read_csv(icpms_file, sep=';',header=1)

print(icpms.columns)

metals = ['55Mn', '59Co','60Ni','66Zn','111Cd','208Pb','63Cu', '56Fe']

#pdf = PdfPages('/Users/christiandewey/Library/CloudStorage/Box-Box/data/assignments/spring/eics_cu_100mz_fullmz-dec13.pdf')

fig, ax = plt.subplots()
for me in metals:
    me_key = 'Time %s' %me

    x = icpms[me_key]/60
    inds = np.asarray((x>=14) & (x<=16))

    x_sub = x[inds]

    y = icpms[me]
    y_sub = y[inds]
          

    x_sub = np.asarray(x_sub).reshape((-1, 1))


    # Create linear regression object
    regr = linear_model.LinearRegression()

    # Train the model using the training sets
    regr.fit(x_sub, y_sub)

    # Make predictions using the testing set
    pred = regr.predict(x_sub)
    rmse=np.sqrt(mean_squared_error(y_sub, pred))
    # The mean squared error
    print(me)
    print("Root mean squared error: %.2f" % rmse)
    # The coefficient of determination: 1 is perfect prediction
    print("3 X RMSE: %.2f" % (3*rmse))

    #ax.plot(icpms[me_key]/60, icpms[me]/1000, label=me)        
    ax.plot(x_sub, y_sub, label=me) 
    ax.plot(x_sub,pred,color='black')       
    maxicp = max(maxicp,max(icpms[me].values))


ax.set_ylim(0,maxicp*1.1)
ax.set_title('ICP-MS Traces (Spring Sample)')
ax.set_xlabel('Time (min)')
ax.set_ylabel('ICP-MS signal (1000 cps)')
ax.legend(bbox_to_anchor=(1.05,0.5),loc='center left',frameon =False)
ax.set_xlim(13.5,16.5)
ax.set_ylim(0,0.1e4)
fig.tight_layout()
plt.show()
